# sp나 통계적인 숫자 이하의 종 떨굴 필요 x
-> 어차피 피쳐로 들어가는 종들만 골라서 진행하는 작업이기 때문

In [15]:
import pandas as pd
from tqdm import tqdm 
from haversine import haversine

In [16]:
df = pd.read_csv('../../20220209_07_21_무당벌레관측.csv')
df2 = pd.read_csv('./ursina_조사.csv')

In [17]:
df2 = df2[df2.Species=='Brachiacantha ursina'].reset_index(drop=True)

In [18]:
df = df[df.public_positional_accuracy<=1000].reset_index(drop=True)

In [19]:
df = df[(df.Latitude!=0.0)&(df.Longitude!=0.0)].reset_index(drop=True)

In [20]:
df.Latitude = df.Latitude.astype(float)
df.Longitude = df.Longitude.astype(float)

df2.Latitude = df2.Latitude.astype(float)
df2.Longitude = df2.Longitude.astype(float)

In [21]:
df_gps = []
for i in tqdm(range(len(df))):
    df_gps.append( (df.Latitude[i], df.Longitude[i]) )
    
    
df2_gps = []
for i in tqdm(range(len(df2))):
    try:
        df2_gps.append( (df2.Latitude[i], df2.Longitude[i]) )
    except:
        print(i)

100%|█████████████████████████████████████████████████████████████████████████████| 769/769 [00:00<00:00, 69900.52it/s]


In [22]:
df['gps'] = df_gps
df2['gps'] = df2_gps

In [23]:
df2=df2[['Species','gps','Year']]

In [24]:
df_dict = {'year' + str(i): df.loc[df["Year"]==i].sort_values(by="Species").reset_index(drop=True) 
           for i in range(2007,2022)}  

In [25]:
# bipunctata (minmax) PTYPE
# ['Lanius_borealis_18', 'Harmonia_axyridis_18',
#        'Cinclus_mexicanus_18', 'Coccinella_septempunctata_18',
#        'Cycloneda_polita_18', 'Eremophila_alpestris_18',
#        'Coelophora_maculata_18', 'Hippodamia_convergens_18',
#        'Hippodamia_tredecimpunctata_18', 'Anatis_mali_18',
#        'Lanius_ludovicianus_18', 'Hippodamia_variegata_18',
#        'Cycloneda_munda_18', 'Mulsantina_picta_18',
#        'Phainopepla_nitens_18', 'Anatis_lecontei_18',
#        'Centronyx_henslowii_18', 'Calvia_quatuordecimguttata_18',
#        'Olla_vnigrum_18', 'Coccinella_novemnotata_18']

In [26]:
bass = ['Coccinella septempunctata',
'Plectrophenax nivalis',
'Hippodamia variegata',
'Harmonia axyridis',
'Coelophora maculata',
'Eremophila alpestris',
'Anatis mali',
'Chilocorus stigma',
'Coccinella trifasciata',
'Cinclus mexicanus',
'Cycloneda polita',
#'Coleomegilla maculata',
'Adalia bipunctata',
'Myzia pullata',
'Anatis labiculata']


PTYPE = []

In [27]:
for i in bass:
    if i in df.Species.unique():
        PTYPE.append(i)

In [37]:
PTYPE=['Coleomegilla maculata']

In [38]:
for i in PTYPE:
    for j in range(2007,2022):            
        globals()['{}_{}df'.format(i.split(" ")[1],j)] = df_dict['year' + str(j)].loc[df_dict['year' + str(j)].Species==i].reset_index(drop=True)

In [39]:
for i in range(2007,2022):
    for j in range(len(df2)):
        for k in PTYPE:
            globals()['{}_{}_{}'.format(k.split(" ")[1],i,j)] = 0   

In [40]:
for i in range(2007,2022):
    for j in PTYPE:
        globals()['{}list_{}'.format(j.split(" ")[1],i)] = []

In [41]:
for i in tqdm(range(len(df2.gps))):
    for j in range(2007,2022):
        for name in PTYPE:
            for k in range(len(globals()['{}_{}df'.format(name.split(" ")[1],j)])):
                if haversine(globals()['{}_{}df'.format(name.split(" ")[1],j)].gps[k], df2.gps[i], unit='km') <= 18:
                    globals()['{}_{}_{}'.format(name.split(" ")[1],j,i)] += 1

100%|███████████████████████████████████████████████████████████████████████████████| 769/769 [00:03<00:00, 235.16it/s]


In [43]:
for name in PTYPE:
    for j in range(2007,2022):
        for k in range(len(df2.gps)):
            globals()['{}list_{}'.format(name.split(" ")[1],j)].append(globals()['{}_{}_{}'.format(name.split(" ")[1],j,k)])

In [47]:
df2

,Species,gps,Year,septempunctata_2007_18km,septempunctata_2008_18km,septempunctata_2009_18km,septempunctata_2010_18km,septempunctata_2011_18km,septempunctata_2012_18km,septempunctata_2013_18km,...,gillamaculata_2012_18km,gillamaculata_2013_18km,gillamaculata_2014_18km,gillamaculata_2015_18km,gillamaculata_2016_18km,gillamaculata_2017_18km,gillamaculata_2018_18km,gillamaculata_2019_18km,gillamaculata_2020_18km,gillamaculata_2021_18km
0,Brachiacantha ursina,"(42.459085, -71.275566)",2007,1,2,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,Brachiacantha ursina,"(42.31722259521485, -71.00861358642578)",2007,6,5,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,Brachiacantha ursina,"(43.0767029177, -89.5210551295)",2007,1,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,1
3,Brachiacantha ursina,"(42.317222, -71.008611)",2007,6,5,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,Brachiacantha ursina,"(42.028619, -94.85571)",2007,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764,Brachiacantha ursina,"(46.6096305847, -72.7027511597)",2021,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
765,Brachiacantha ursina,"(45.4816548963, -71.8159053102)",2021,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
766,Brachiacantha ursina,"(45.4273762997, -75.7640765997)",2021,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
767,Brachiacantha ursina,"(45.02695, -74.7129366667)",2021,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [45]:
for name in PTYPE:
    for j in range(2007,2022):
         df2["gilla"+name.split(" ")[1]+"_"+str(j)+"_18km"] = globals()['{}list_{}'.format(name.split(" ")[1],j)]



In [ ]:
df2

In [51]:
df2.to_csv('./ursina_실전테스트_무당벌레.csv')